## Environment setup

### Library preparation

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
from si_prefix import si_format
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
# from qcodes import Station, load_by_run_spec, load_by_guid
# from qcodes.instrument.base import Instrument
# from qcodes.dataset.experiment_container import (Experiment,
#                                                  load_last_experiment,
#                                                  new_experiment)
# from qcodes.dataset.database import initialise_database
# from qcodes.dataset.measurements import Measurement
# from qcodes.dataset.plotting import plot_by_id, get_data_by_id, plot_dataset
# from qcodes.dataset.data_set import load_by_id
# # from qcodes.dataset.data_export import get_shaped_data_by_runidb

In [3]:
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.tektronix.AWG3252_Isrc import AWG3252_Isrc
from qcodes.instrument_drivers.HP.HP34401 import HP34401
from qcodes.instrument_drivers.HP.HP34401_DL1201 import HP34401_DL1201
from qcodes.instrument_drivers.Keithley.Keithley_2600_channels import *
from qcodes.instrument_drivers.tektronix.AWG5204_Isrc import AWG5204_Isrc

from qcodes.instrument_drivers.Keysight.Keysight_B2962A_Isrc import B2962A_Isrc


from qcodes.instrument_drivers.Keithley.Keithley_6220 import Keithley_6220
from qcodes.instrument_drivers.Keithley.Keithley_6220_Isrc import Keithley_6220_Isrc

In [4]:
import sys 
sys.path.append('..\..\_jjtools')

# from Exps import *

from JJ_data_processing import read_opj_data, pbi, bpbi, show_df, update_df

from JJformulas import *
# from meas_util import *

from JJmeasurement import *

C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'
C:\Users\KelvinOX25\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Instrument nicknames and QC database initialization

In [653]:
try:
   Instrument.close_all()
except KeyError:
   pass    
except NameError:
   pass 

K2602 =  Keithley_2600(name = 'K2602', address = "GPIB::26::INSTR" )
Flux_chA = K2602.smua.curr
Flux_chB = K2602.smub.curr

Bfield =  Flux_chA
Bfield.step = 10e-6
Bfield.inter_delay = .1



# Isrc = AWG5204_Isrc('gen', 'TCPIP0::AWG52000-XXXX::inst0::INSTR', ch = 1, R_bias = 1e9, Attn = 1, timeout = 20)


# Isrc = Keithley_6220(name = 'K6220', address = "GPIB::13::INSTR" )
# Isrc.set_R_Attn ( R_bias = 1, Attn = 1 )




# Isrc = AWG3252_Isrc('gen', 'TCPIP0::192.168.13.32::inst0::INSTR', 
#                     R_bias = 1e9, Attn = 10)
# Isrc.init()

Vmeter = HP34401_DL1201('meter', 'GPIB0::8::INSTR', Gain = 1e2)
Vmeter.init('fast 6')


# Isrc = Keithley_6220_Isrc(name = 'K6220', address = "GPIB::13::INSTR",
#                           Rsh = 10e6, Rb = 1e9, Vmeter = Vmeter)


Isrc = Keithley_6220_Isrc(name = 'K6220', address = "GPIB::13::INSTR",
                          Rsh = 10.041e3, Rb = 2.2e6, Vmeter = Vmeter)


station = qc.Station(Isrc, Vmeter)

print ('Instrument initialization completed.')

Connected to: Keithley Instruments Inc. 2602A (serial:1272049, firmware:2.1.5) in 0.11s


[K6220(Keithley_6220_Isrc)] Snapshot: Could not update parameter: Irange


Instrument initialization completed.


In [654]:

LS370 = Model_372(name = 'LS370 input Ch', address = 'GPIB::12::INSTR', terminator='\n')
T8 = LS370.ch08.temperature

htr = LS370.sample_heater
htr_setpt = htr.setpoint

htr_setpt.set(0.020)
htr.P.set(30)
htr.I.set(5)
htr.D.set(1)
# PIDset(30,5,1)
htr.output_range('3.16mA')

Connected to: LSCI 370 (serial:370665, firmware:04102008) in 0.15s


### Class JJmeas

In [655]:
tools = {'I' : Isrc.I,
         'V' : Vmeter.V,
         'B' : Flux_chA,
         'T': T8,
         'htr' : htr_setpt}

In [656]:
def setup():
    Isrc.I.set(0)
#     pass

def cleanup():
    Isrc.I.set(0) 
    Bfield.set(0)



In [725]:
jj = JJmeas(sample = 'D070B1N1v1', tools = tools)
jj.setup = setup
jj.cleanup = cleanup

jj.ZF   = -2e-3
jj.FF   = 0.200e-3

In [727]:
jj1 = JJmeas(sample = 'D070B1N1v1', tools = tools)
jj1.setup = setup
jj1.cleanup = cleanup


jj5 = JJmeas(sample = 'D070B1N5v1', tools = tools)
jj5.setup = setup
jj5.cleanup = cleanup

jj0 = JJmeas(sample = 'D070B1N0v1', tools = tools)
jj0.setup = setup
jj0.cleanup = cleanup

In [704]:
# jj = JJmeas(sample = 'D078N5v1', tools = tools)
# jj.setup = setup
# jj.cleanup = cleanup

In [541]:
jj.tool_status(['B', 'I'])

{'B': '-93.0 µA', 'I': '-338.3 fA'}

In [186]:
#values without extra shield

# jj.ZF   = -0.093e-3
# jj.FF   = 0.130e-3

## Measurement

In [542]:
tools['I'].set(1e-10)

In [543]:
tools['I'].get()

9.904516115311889e-11

#### Iscr choise

In [544]:
Isrc.ask_raw('SOUR:CURR:RANG?')

'2.000000E-07\n'

In [720]:
Isrc.write_raw('SOUR:CURR:RANG 1e-7')
Isrc.I.set(0.0e-9)

In [602]:
Isrc.I.set(0e-9)

#### Vmeter regime

In [18]:
Vmeter.init('fast 6')

In [152]:
jj.meas_Voffset(0)

5.9592217e-05

In [162]:
jj.tools['V'].meas_Voff()
Isrc.Vmeter.Voff

5.847299300000001e-05

In [496]:
B = jj.cos_to_B(1)

print(B*1e3)

Bfield.set(B)


-0.093


### IVCs

In [614]:
jj.stabilize_I( amp = 200e-12)

#### wide scan

In [717]:
#B = jj.cos_to_B(0)

B = 0

print(B*1e3)

Bfield.set(B)


jj.IVC_udu (amp = 100e-9, stp = 1e-9, dt = .1,  N_avg = 1, 
            label = 'kei full ivc')

0.0


Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|█████████████| 1/1 [00:00<00:00, 249.99it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|█████████████| 1/1 [00:00<00:00, 200.00it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|█████████████| 1/1 [00:00<00:00, 249.99it/s]
Starting experimental run with id: 1


1

#### fine scan

In [744]:
# B = jj.cos_to_B(0)
B = -380e-6

print(B*1e3)

Bfield.set(B)


jj0.IVC_udu (amp = 500e-12, stp = 1e-12, dt = .1,  N_avg = 1, 
            label = 'kei gnd B = -380')

-0.38


Starting experimental run with id: 75


75

In [629]:
imin = .5e-12

i_list =   np.concatenate ([np.linspace(0    , 35e-12, 21), 
                            np.linspace(12e-12, imin, 21),
                            np.linspace(imin, 12e-12, 251),
                            np.linspace(12e-12,  0e-12, 21)]) 

jj.IVC_cust ( i_list, Ioff = 0, dt = 0.1, N_avg = 1, label = 'B=0m intermid steps')

Starting experimental run with id: 1387


An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-0173770821bc;
Traceback:
Traceback (most recent call last):
  File "..\..\_jjtools\JJmeasurement.py", line 82, in IVC_cust
    is_vs = [[I.get(),V.get()] for _ in range( N_avg)]
  File "..\..\_jjtools\JJmeasurement.py", line 82, in <listcomp>
    is_vs = [[I.get(),V.get()] for _ in range( N_avg)]
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\instrument\parameter.py", line 448, in get_wrapper
    value = get_function(*args, **kwargs)
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\utils\command.py", line 178, in __call__
    return self.exec_function(*args)
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\utils\command.py", line 127, in call_by_str
    return self.exec_str(self.cmd_str.format(*args))
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\instrument\base.py", line 759, in ask
    answer = self.ask_raw(cmd)
  File "C:\Users\KelvinOX25\Anaconda3\lib\

KeyboardInterrupt: 

### Ic meas

In [572]:
B = 0e-6
Bfield.set(B)

i_list = np.linspace(0, 200e-12, 101)

jj.IVC_cust (i_list, Vthr = 30e-6, dt = 0.1,
             N_avg = 1, label = 'Kei Ic highRb RT cos = 1')

Starting experimental run with id: 1185


1185

### Time scan

In [513]:
# Bfield.set(0e-3)
jj.time_scan ( tools['I'], dur = 30, dt= 0.1)

Starting experimental run with id: 1093



1093

In [148]:
B_list = np.linspace(0e-3, .3e-3, 51)

[            jj.time_scan ( tools['V'], dur = 1, dt= 0.01)
for jj in    jj.Bscan( B_list = B_list )]

Starting experimental run with id: 372



Starting experimental run with id: 373



Starting experimental run with id: 374



Starting experimental run with id: 375



Starting experimental run with id: 376



...

Starting experimental run with id: 419



Starting experimental run with id: 420



Starting experimental run with id: 421



Starting experimental run with id: 422




[372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422]

### IVC B

#### New syntaxis to try

In [624]:
# B_list = udu_list ( 4e-3, 0.2e-3)
B_list = np.linspace(-.10e-3, 0.5e-3, 121)
# cos_list = np.linspace(1,0, 26)

[            jj.IVC_udu (amp = 100e-12, stp = 1e-12, dt = .1,
                         N_avg = 1, label = 'fine B scan')
for jj in    jj.Bscan( B_list = B_list )]
# for jj in    jj.Bscan( cos_list = cos_list )]


Bfield.set(0)


Starting experimental run with id: 1212


An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-017373327a5c;
Traceback:
Traceback (most recent call last):
  File "..\..\_jjtools\JJmeasurement.py", line 82, in IVC_cust
    is_vs = [[I.get(),V.get()] for _ in range( N_avg)]
  File "..\..\_jjtools\JJmeasurement.py", line 82, in <listcomp>
    is_vs = [[I.get(),V.get()] for _ in range( N_avg)]
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\instrument\parameter.py", line 452, in get_wrapper
    value = self.get_parser(value)
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\instrument_drivers\Keithley\Keithley_6220_Isrc.py", line 91, in get_I
    v = self.Vmeter.V.get() - Voff
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\instrument\parameter.py", line 448, in get_wrapper
    value = get_function(*args, **kwargs)
  File "C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\utils\command.py", line 178, in __call__
    return self.exec_function(*args)
  File "C:\Users\

KeyboardInterrupt: 

#### Ic(B)

In [ ]:
i_list = np.linspace(0, 500e-12, 501)
B_list = np.linspace(-1e-3, 1e-3, 201)
# cos_list = np.linspace(1,0, 11)

jj = jj0

[            jj.IVC_cust ( i_list, Vthr = 50e-6, dt = 0.1, N_avg = 1, label = 'fine Ic(B)')
for jj in    jj.Bscan( B_list = B_list )]


Bfield.set(0)

Starting experimental run with id: 76


Starting experimental run with id: 77


Starting experimental run with id: 78


In [113]:
i_list = np.concatenate([np.linspace(0, 30e-12, 251),
                         np.linspace(30e-12, 0,  251)]) 
B_list = np.linspace(-1e-3, 1e-3, 51)

[            jj.IVC_cust ( i_list, dt = 0.1, N_avg = 1, label = 'test Ic(B)')
for jj in    jj.Bscan( B_list = B_list )]


Bfield.set(0)

Starting experimental run with id: 241


Starting experimental run with id: 242


Starting experimental run with id: 243


Starting experimental run with id: 244


Starting experimental run with id: 245


...

Starting experimental run with id: 288


Starting experimental run with id: 289


Starting experimental run with id: 290


Starting experimental run with id: 291



AttributeError: ("'Model_372' object and its delegates have no attribute 'visa_log'", "asking 'RDGK? 8\\n\\r' to <Model_372: LS370 input Ch>", 'getting LS370 input Ch_ch08_temperature')

In [174]:
# B_list = udu_list ( 4e-3, 0.2e-3)
B_list = np.linspace(0, 1.85e-3, 11)

i_list = udu_list(1e-12, 0.01e-12) -.25e-12



[            jj.IVC_cust (i_list, dt = .5, N_avg = 4)
for jj in    jj.Bscan( B_list = B_list )]

Bfield.set(0)


Starting experimental run with id: 44


Starting experimental run with id: 45


Starting experimental run with id: 46


Starting experimental run with id: 47


Starting experimental run with id: 48


...

Starting experimental run with id: 50


Starting experimental run with id: 51


Starting experimental run with id: 52


Starting experimental run with id: 53


Starting experimental run with id: 54



In [230]:
jj.ZF = 0
jj.FF = 1.85e-3
cos_list = [1,0]

[            jj.IVC_fwd (amp = 1e-12, stp = 0.1e-12, dt = .5, Ioff = -.35e-12)
for jj in    jj.Bscan( cos_list = cos_list )]

Starting experimental run with id: 63


Starting experimental run with id: 64

{ 'ids' : range(63,64+1), 'B' : np.linspace(0.00e+00,1.85e-03, 2), 'T' : 0.030, 'comm : '' }


[63, 64]

### T scan

In [210]:
5*11*6/60

5.5

In [135]:
htr.HTRset('3.16mA')

'Heater changed = 3.16mA'

In [150]:
htr.output_range('3.16mA')

In [109]:
T8.get()

0.0501275

In [ ]:
[            jj.IVC_fwd (amp = 1e-12, stp = 0.025e-12, dt = .5, N_avg = 100)
for jj in    jj.Tscan( [200e-3] )]

htr.set(0.030)

In [19]:
np.array( [50 + 25*i for i in range (11) ] )*1e-3

array([0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  , 0.225, 0.25 ,
       0.275])

In [21]:
htr_setpt.set(30e-3)

In [13]:
htr.output_range('3.16mA')

In [28]:
T_list = np.array( [50 + 25*i for i in range (11) ] )*1e-3
B_list = np.linspace(0, 1.85e-3, 11)

for jj in  jj.Tscan( T_list ):
    
    for jj in jj.Bscan(B_list):
    
        jj.IVC_udu (amp = 1e-12, stp = 0.01e-12, dt = .5, Ioff = -.5e-12)
    
    
Bfield.set(0e-3)
htr_setpt.set(30e-3)

ramping T8 to 50.0 mK...
50.9 m
50.7 m
50.7 m
50.9 m
51.0 m
50.5 m
50.5 m
50.3 m
50.6 m
50.2 m
50.2 m
50.2 m
49.9 m
49.6 m
49.8 m
50.1 m
50.0 m
49.6 m
49.7 m
49.5 m
49.7 m
49.6 m
T is set


Starting experimental run with id: 912


Starting experimental run with id: 913


Starting experimental run with id: 914


Starting experimental run with id: 915


...

Starting experimental run with id: 919


Starting experimental run with id: 920


Starting experimental run with id: 921


Starting experimental run with id: 922

{ 'ids' : range(912,922+1), 'B' : np.linspace(0.00e+00,1.85e-03, 11), 'T' : 0.050, 'comm : '' }
ramping T8 to 75.0 mK...


VisaIOError: ('VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.', "asking 'RDGK? 8\\n\\r' to <Model_372: LS370 input Ch>", 'getting LS370 input Ch_ch08_temperature')

In [ ]:
T_list = np.array( [200 + 25*i for i in range (5) ] )*1e-3
B_list = np.linspace(0, 1.85e-3, 11)

for jj in  jj.Tscan( T_list ):
    
    for jj in jj.Bscan(B_list):
    
        jj.IVC_fwd (amp = 2e-12, stp = 0.04e-12, dt = .5, Ioff = -.35e-12)
    
    
Bfield.set(0e-3)
htr_setpt.set(30e-3)